In [3]:
#import dpkt
#import matplotlib.pyplot as plt
#import numpy
#import math
# import pickle5 as pickle
import pickle # für Obj serialisation
#import numpy as np
#import matplotlib
import os
import time
from datetime import datetime
import json
#import csv
from pathlib import Path

In [4]:
TransitionPeriod = 4
amazon_ips = {} # Amazon meint wohl ob eingehend oder ausgehend
GAMMA = 0.25
observation_lengths = [180, 300, 900, 1800] # in seconds (3min - 30min)
LENGTH = 3600 #
"""Eq 3600"""
MAGIC_IsUserPacketARelevantBurstPacket = 40 # Wird bei der Event Extraction verwendet... Wenn größer, dann wird Paket als Relevantes Burst Paket verwendet.

In [5]:
DELTA = 15 # seconds -> Maximale Zeitliche Differenz.. Wird schrittweise erhöht 
size_threshold = 10000

## Parameter aus Paper:
- t_e = 0.5s (Burst detection Threshold) 
- Delta = 3s (Abweichung der Event Times)
- Γ (Gamma) = 10kb (Abweichung der BurstGröße)

 
----


In [ ]:
# Reads files and the Preprocessed 

def get_message_file_names(rangeBegin, rangeEnd, messages_dir):
    """Copys All messages contained in \"all_messages_path\" to arg \" messages_dir\""""

    all_messages_path = 'path/to/generated_messages/messages_pareto_xm_5000_alpha_0.93_max30delay_ver02' # Message Traces ?
    all_messages_file_names = sorted(os.listdir(all_messages_path))
    if not os.path.exists(messages_dir):
        os.makedirs(messages_dir)
    for i in range(len(all_messages_file_names)):
        if i >= rangeBegin and i <= rangeEnd:
            print(all_messages_file_names[i])
            os.system('cp {} {}'.format(os.path.join(all_messages_path, all_messages_file_names[i]), os.path.join(messages_dir,all_messages_file_names[i])))
    return


root_dir = 'path/to/root/directory/of/traces' # Root folder of Traces


timestamps_dirs = []
messages_dirs = [] # Generated Message Traces -> Concrete Sub folder
message_file_names = []
packets_pickle_dirs = []
packets = []
dir_count = 0
for path in sorted(Path(root_dir).iterdir()): # for every file in RootDir

    if path.is_dir() and path.name != 'aggregate' and not path.name.endswith('extra'): # if is Directory        
        print(path)

        timestamps_dirs.append(os.path.join(path, 'timestamps')) # fügt Timestamp Paths zu Timestamp Ordner hinzu
        print('timestamps are in {}'.format(timestamps_dirs[-1]))

        messages_dir = Path(os.path.join(path, 'messages')) # fügt dem Message ordner die konkreten Pfad hinzu
        messages_dir.mkdir(parents=True, exist_ok=True) # kp wieso dafür ein neuer ordner erstellt wird ?
        messages_dirs.append(os.path.join(path, 'messages')) # fügt ein neuen message pfad dem messages_dirs hinzu        
        print('messages are in {}'.format(messages_dirs[-1]))

        range_begin = int(path.name.split('_')[0])
        range_end = int(path.name.split('_')[1])
        print(range_begin, range_end)

        get_message_file_names(range_begin, range_end, messages_dirs[dir_count])
        message_file_names.extend(sorted(os.listdir(messages_dirs[dir_count])))
        print('length of message_file_names is {}'.format(len(message_file_names)))

        packets_pickle_dirs.append(os.path.join(path, 'traces/pickles/packets'))


        # Reading in preprocessed Packets (serialized with JSON)
        # read all packets from packets pickle and write them to 'packets'
        pickle_name = os.listdir(packets_pickle_dirs[dir_count])[0]
        print('loading {} ...'.format(pickle_name))
        start = time.time()

        with open(os.path.join(packets_pickle_dirs[dir_count], pickle_name[:-7] + '.pickle'), 'rb') as handle: #Ist wie try catch with Ressource (handle)
            packets.extend(pickle.load(handle))

        end = time.time()
        hours, rem = divmod(end - start, 360)
        mins, seconds = divmod(rem, 60)

        print('Pickle loading took {:0>2}:{:0>2}:{:05.2f}'.format(int(hours), int (mins), seconds))

        dir_count += 1
aggregate_dir = Path(root_dir) / 'aggregate'
aggregate_dir.mkdir(parents=True, exist_ok=True)
results_dir = str(aggregate_dir)

In [ ]:
timestamps_of_all_senders = []
"""
### Array of timestamps array (grouped by files?)
"""
message_sizes_of_all_senders = []
message_types_of_all_senders = []
for i in range(dir_count):
    timestamps_of_all_senders.extend(get_timestamps_of_all_senders(messages_dirs[i], timestamps_dirs[i]))
    message_sizes_of_all_senders.extend(get_sizes_of_all_senders(messages_dirs[i]))
    message_types_of_all_senders.extend(get_types_of_all_senders(messages_dirs[i]))

In [ ]:
packets[500] # Should generate IllegalAcces.. (Should not be set)

{'to_amazon': False,
 'src': '192.168.122.91',
 'dst': '192.168.122.75',
 'size': 64,
 'timestamp': 1634137925.66455,
 'protocol': 6,
 's_port': 1080,
 'd_port': 40940}

In [7]:
def get_sizes_sender(messages_file, max_count=None):
    """
    Returns the Sizes (as Array) from Message Traces JSON
    -> Loads Message File (JSON)
    """
    sizes = []
    with open(messages_file) as f:
        data = json.load(f) # die Generated Message Traces
        count = 0
        for message in data['messages']:
            if max_count is None or count < max_count:
                sizes.append(int(message['size'])) # sizes are in bytes
            else:
                break
            count += 1
    return sizes
    
def get_timestamps_sender(timestamps_file, max_count=None):
    """
    Returns an Array of Timestamps from the Timestamps file (list of Tmstmps an IDs)
    """
    timestamps = []
    with open(timestamps_file) as f:
        lines = f.readlines()
        count = 0
        for line in lines:
            if max_count is None or count < max_count:
                line = line.split()
                timestamps.append((float)(line[2])) # Nimmt den Timestamp (spalte 3)
            else:
                break
            count += 1
    return timestamps

def get_sizes_of_all_senders(messages_dir):
    """
    Returns the Sizes as Array from all Message traces Files in messagesdir
    """
    message_file_names = sorted(os.listdir(messages_dir))
    all_message_sizes_of_senders = []
    for mf_name in message_file_names:
        message_sizes = get_sizes_sender(os.path.join(messages_dir, mf_name))
        all_message_sizes_of_senders.append(message_sizes)
    return all_message_sizes_of_senders

def get_timestamps_of_all_senders(messages_dir, timestamps_root_dir):
    """
    Returns all corresponding timestamps of messages_traces... (it takes the timestamp file corresponding to the Messages file from messages_dir)
    As Array of Timestamps Array
    """
    message_file_names = sorted(os.listdir(messages_dir))
    all_timestamps = []
    for mf_name in message_file_names:
        timestamps = get_timestamps_sender(os.path.join(timestamps_root_dir, 'timestamps_' + mf_name[:-5] + '.txt'))
        all_timestamps.append(timestamps)
    return all_timestamps

def get_types_of_all_senders(messages_dir, max_count=None):
    """
    Return all Types as Array from all Messages in Messages dir (goes through all files and every message in each file)
    """
    message_file_names = sorted(os.listdir(messages_dir))
    all_types = []
    for messages_file in message_file_names:
        with open(os.path.join(messages_dir, messages_file)) as f:
            types = []
            data = json.load(f)
            count = 0
            for message in data['messages']:
                if max_count is None or count < max_count:
                    types.append((message['type']))
                else:
                    break
                count += 1
            all_types.append(types)
    return all_types

In [ ]:
def separate_receiver_packets_non_chronological_traces(packets, timestamps_of_all_senders):
    """
    Returns a Array containing Arrays of Packets grouped by Timestamp files (with padding)
    """
    packets_of_all_receivers = []
    all_beginnings = []
    all_endings = []
    for sender_timestamps in timestamps_of_all_senders:
        """
        Erstellt Array von Allen Start und Ends der Timestamp files (mit puffer)
        """
        beginning = sender_timestamps[0] - 5 #seconds <-- offset um früher zu beginnen (die Timestamps sind eigentlich in us aber  mit kommazahl.. deswegen sekunde)
        ending = sender_timestamps[len(sender_timestamps)-1] + 30 #seconds
        all_beginnings.append(beginning) # Array aller Timestamp files Starts - 5 Sekunden
        all_endings.append(ending) # Array aller Timestamp files Endes + 30 Sekunden
    print(all_beginnings)
    print(all_endings)

    current_sender = 0
    num_of_senders = len(timestamps_of_all_senders) # Das macht keinen sinn für mich.. 
    #Sind nicht die anzahl der Sender sondern die Anzahl der Timestamp files! Und die sind einfach nur unterteilt (fortlaufende timestamps und IDs)
    current_sender_received_packets = []
    flag = 0
    for p in packets:
        if p.get('timestamp') > all_beginnings[current_sender] and p.get('timestamp') < all_endings[current_sender]:
            current_sender_received_packets.append(p) # Wenn Packet innerhalb des Zeitbereichs des Timestamp files ist (hier als Sender) wird es zu den empfangenen Paketen hinzugefügt

        elif current_sender == num_of_senders - 1: # Wenn am ende der Timestamp files (hier sender)
            packets_of_all_receivers.append(current_sender_received_packets) # Füge das Array der pakete (dieses Timestamp files) dem Gesammtarray hinzu
            break
        elif p.get('timestamp') >= all_beginnings[current_sender + 1] and p.get('timestamp') < all_endings[current_sender + 1]: # Wenn Packet innerhalb des Zeitbereichs des nächsten Timestamp files liegt (hier sender)
            packets_of_all_receivers.append(current_sender_received_packets)
            current_sender += 1 # Wähle das nächste Timestamp file aus (hier sender)
            current_sender_received_packets = []
            current_sender_received_packets.append(p)
    return packets_of_all_receivers

def separate_receiver_packets_chronological_traces(packets, timestamps_of_all_senders):
    """
    Same as non Chronological ? @see separate_receiver_packets_non_chronological_traces
    """
    packets_of_all_receivers = []
    all_beginnings = []
    all_endings = []
    for sender_timestamps in timestamps_of_all_senders:
        beginning = sender_timestamps[0] - 5 #seconds
        ending = sender_timestamps[len(sender_timestamps)-1] + 30 #seconds
        all_beginnings.append(beginning)
        all_endings.append(ending)
    print(all_beginnings)
    print(all_endings)
    current_sender = 0
    num_of_senders = len(timestamps_of_all_senders)
    current_sender_received_packets = []
    for p in packets:            
        if p.get('timestamp') > all_beginnings[current_sender] and p.get('timestamp') < all_endings[current_sender]:
            current_sender_received_packets.append(p)
        elif p.get('timestamp') >= all_endings[current_sender]:
            packets_of_all_receivers.append(current_sender_received_packets)
            if current_sender < num_of_senders - 1:
                current_sender += 1
                current_sender_received_packets = []
                if p.get('timestamp') > all_beginnings[current_sender] and p.get('timestamp') < all_endings[current_sender]:
                    current_sender_received_packets.append(p)
            else:
                break
    return packets_of_all_receivers

In [ ]:
packets_of_all_receivers = separate_receiver_packets_non_chronological_traces(packets, timestamps_of_all_senders)
packets_towards_receivers = []
"""
Contain Arrays with the Packets grouped by the Timestamp files
"""
packets_towards_receivers = [[p for p in packets if (p.get('to_amazon') is False)] for packets in packets_of_all_receivers] # Ultra hässlich.. -> Selectiert alle packete die "to Amazon" = false sind

In [ ]:
def convert_time(timestamps, start_time):
    """
    Converts timestamps to relative timestamps (relative to Startime)
    """
    converted_timestamps = []
    for t in timestamps:
        converted = t - start_time
        if converted < 0:
            print('Error: timestamp is larger than start time with t = {} and start_time = {}'.format(t, start_time))
        converted_timestamps.append(converted)
    return converted_timestamps    

def get_on_periods_of_sender_2(timestamps, message_sizes):
    """
    Returns an Array of timestamp and messagesSizes Tupels\\
    eg.   
     `[(timestamps[0], messagesizes[0]), (timestamps[1], messagesizes[1]).... ]`
    """
    global TransitionPeriod # Macht doch gar nichts ... können wir auch weglöschen?
    bursts = []
    timestamps_Sizes_tupel = zip(timestamps, message_sizes) # Creates a Tupel... (timestamps[0], messagesizes[0])...
    for t, s in timestamps_Sizes_tupel:
        bursts.append((t, s))
    return bursts

""" def get_on_periods_of_user(timestamps, user_packet_sizes): # Wird nicht benutzt
    global TransitionPeriod
    is_in_burst = False
    bursts = []
    burst_size = 0
    last_burst_index = 0
    for time_iter in range(0, len(timestamps)):
        if time_iter == len(timestamps) - 1 and is_in_burst == True:
            if burst_size > 1514:
                bursts.append((timestamps[last_burst_index], burst_size))
                break
        if timestamps[time_iter] - timestamps[last_burst_index] > TransitionPeriod:
            if is_in_burst == True:
                if burst_size > 1514:
                    bursts.append((timestamps[last_burst_index], burst_size))
                is_in_burst = False
                burst_size = 0
        if user_packet_sizes[time_iter] > 1400:
            if is_in_burst == False:
                is_in_burst = True
                last_burst_index = time_iter
            burst_size += user_packet_sizes[time_iter]
    return bursts


def get_bursts_user_2(timestamps, user_packet_sizes): # Wird nicht benutzt
    bursts = []
    burst_size = 0
    for time_iter in range(0, len(timestamps) - 1):
        ipd = timestamps[time_iter + 1] - timestamps[time_iter]
        if ipd < 1:
            if user_packet_sizes[time_iter] > 40:
                burst_size += user_packet_size[time_iter]
        else:
            if burst_size > 20000:
                bursts.append((timestamps[time_iter], burst_size))
            burst_size = 0
    return bursts """

def get_bursts_user(timestamps, user_packet_sizes):
    """
    Returnes A List of Tupels containing the Timestamps and the Packet Burstsizes (Filters only Correct burst (40bytes threshold))
    Merges Packets together that belong to one Burst (one Second ?) some Magic constant -> Should be the T_e Parameter
    """
    global TransitionPeriod
    is_in_burst = False
    bursts = []
    burst_size = 0
    last_burst_index = 0
    MTU = 1500 #TODO Constant ist damit wirklich die MTU gemeint ? ... Oder wird damit nur nochmal doppelt geprüft falls leitung mit geringer bandbreite verwendet wird.. Aber dann macht MTU size keinen sinn..

    for time_iter in range(0, len(timestamps)): # Time_iter ist index für jede Nachricht (markiert durch timestamp)
        if time_iter == len(timestamps) - 1 and is_in_burst: # Wenn letzte Msg und wir innerhalb eines Bursts sind ?
            if burst_size > MTU: # Und es ein Burst Paket ist -> Verstehe den MTU parameter nicht so richtig -> Seite 8 unten rechts
                bursts.append((timestamps[last_burst_index], burst_size)) # Speichern wir diesen Burst, so wie auch normal (Nur das hier richtigerweise der LastburstIndex richtig ist)!!!
                break

        if (timestamps[time_iter] - timestamps[last_burst_index]) > 1: # Wenn mehr als eine Sekunde vergangen ist seit ende des letzten Burst -> Sind wir nicht mehr in einem Burst
            if is_in_burst:
                if burst_size > MTU:
                    #TODO is Programmier Fehler hier ? müsste nicht last_burst_index sein? Siehe seite 9 und 6 Zeilen weiter oben
                    bursts.append((timestamps[time_iter], burst_size)) # Speichern des Bursts (als Tupel Timestamp mit der Burstgröße) (Warum nicht den Letzten Burst Timestamp???!!)
                is_in_burst = False #-> Sind wir nicht mehr in einem Burst
                burst_size = 0
        if user_packet_sizes[time_iter] > MAGIC_IsUserPacketARelevantBurstPacket: # magic constant -> Wenn packet size > 40 Bytes (Wahrscheinlich wenn nicht nur steuerungsinformationen) #TODO
            if is_in_burst == False:
                is_in_burst = True # Befinden wir uns in einem Burst
            last_burst_index = time_iter # Setzen des des Bursts (wird immer neu gesetzt -> Markiert das Ende)
            burst_size += user_packet_sizes[time_iter] # Addieren des Bursts
    return bursts



"""def merge_channel_period_points(ChannelPeriodPoints):
    
    #Wird das überhaupt benutzt ?
    
    #Merge Entrys together that are closer then 2 [unit missing? Seconds?]
    #Outputs List of Tupels

    #-> Könnte sein um das mergen durch die Event Extraction (get_bursts_user) nachzubilden..
    #-> Hat aber einen anderen Threshold..
    
    c = []
    time = [x[0] for x in ChannelPeriodPoints] # Schreibt alle times ChannelPeriodPoints[i].getTime() in time
    size = [x[1] for x in ChannelPeriodPoints] # Schreibt alle sizes ChannelPeriodPoints[i].getSize() in Size
    burst_time = []
    burst_volume = []
    burst_volume.append(size[0])
    burst_time.append(time[0])
    temp = time[0]
    for i in range(len(time)-1): # for(int i = 0; i < time.length(); i++)
        if (time[i+1] - temp) < 2: # Wenn abstand zum letzten kleiner als 2 ?? #TODO Magic Constant
            burst_volume[-1] += size[i+1] # [-1] is the last element (wrap around) | Addiert auf das letzte burst_volume die nächste Größe
            temp = time[i+1]
        else:
            burst_time.append(time[i+1]) # Und sonst wenn ausserhalb des bereichs.. Füge weiteres Time ELement hinzu
            burst_volume.append(size[i+1]) # Und weitere größe
            temp = time[i+1]
    for i in range(len(burst_time)):
        c.append((burst_time[i], burst_volume[i]))
    return c
"""

"""def get_channel_points(f): # Wird nicht benutzt
    ChannelPeriodPoints = []
    first_line = True
    for line in f:
        line = line.split()
        if first_line:
            find_start_time(line[1].decode('ascii'))
            first_line = False
            continue
        message_type = line[4].decode('ascii')
        if message_type == 'text':
            continue
        time = convert_time(line[2].decode('ascii'))
        if line[5].decode('ascii') == 'None':
            size = 300000
        else:
            size = int(line[5].decode('ascii'))
        ChannelPeriodPoints.append((time, size))
    if len(ChannelPeriodPoints) == 0:
        return []
    c = merge_channel_period_points(ChannelPeriodPoints)
    return c"""

## Hier wird extraction gemacht glaube ich

In [1]:
all_user_bursts = []
"""

"""
all_channel_bursts = [] # the same as adversary bursts in the case of one-on-one chats

for i in range(0, len(packets_towards_receivers)): # Zählt durch alle Timestamp Gruppen (die Arrays mit den Paketen enthalten)  

    print('Hour {} {}'.format(i, message_file_names[i])) # Warum auch immer das Hour sind ?

    channel_timestamps = timestamps_of_all_senders[i] # Hier wird entpackt... die Timestamps der Nachrichten die Mitbeobachtet wurden im Channel (durch Adversary messaging Bot)
    channel_message_sizes = message_sizes_of_all_senders[i]

    user_packet_timestamps = [p.get('timestamp') for p in packets_towards_receivers[i]] # Alle Timestamps extrahieren in dem konkkreten Timestamp file
    user_packet_sizes = [p.get('size') for p in packets_towards_receivers[i]] # Alle größen extrahieren

    if len(user_packet_timestamps) == 0: # Wenn nichts enthält wird weitergegangen
        all_user_bursts.append([])
        all_channel_bursts.append([])
        print('no receiver timestamps')
        continue
    
    start_time = min(channel_timestamps[0], user_packet_timestamps[0]) # Channel timestamps kommen aus dem Timestamp file - User Packet timestamps aus den Packets .. der start wird ermittelt
    print(channel_timestamps[0])
    print(user_packet_timestamps[0])
    print('start time: {} diff: {}'.format(start_time,abs(channel_timestamps-user_packet_timestamps)))

    converted_sender_timestamps = convert_time(channel_timestamps, start_time) # Sets all timestamps (from timestamp file) relativ to the first of this file
    sender_period_points = get_on_periods_of_sender_2(converted_sender_timestamps, channel_message_sizes) # Creates Tupel list with message sizes and Converted Timestamps

    # Does the same with the Timestamps from the Captured Packets
    converted_user_timestamps = convert_time(user_packet_timestamps, start_time)
    # And gets the Analyzed Bursts with the Timestamps (relativ to the start of this page)
    user_period_points = get_bursts_user(converted_user_timestamps, user_packet_sizes)

    all_user_bursts.append(user_period_points)
    all_channel_bursts.append(sender_period_points)
    print ('Hour {} is done'.format(i))
    print ("\n")

NameError: name 'packets_towards_receivers' is not defined

### Store the Bursts:

In [ ]:
# Stores the Bursts in Pickles 

with open(os.path.join(results_dir, 'all_user_bursts.pickle'), 'wb') as handle:
    pickle.dump(all_user_bursts, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(results_dir, 'all_channel_bursts.pickle'), 'wb') as handle:
    pickle.dump(all_channel_bursts, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Load the Bursts

In [ ]:
with open(os.path.join(results_dir, 'all_user_bursts.pickle'), 'rb') as handle:
    all_user_bursts = pickle.load(handle)
with open(os.path.join(results_dir, 'all_channel_bursts.pickle'), 'rb') as handle:
    all_channel_bursts = pickle.load(handle)

Implement the technique

In [ ]:
def find_intervals(period_points, minute):
    """
    Splits the Period Points list into Time Intervals. Specified by "minute"
    
    `return:` list of periodPoints (wahrscheinlich msgEvents). Jedes Listenelement ist eine Liste mit msgEvents innerhalb eines Intervals
    """
    global LENGTH #=3600
    num_of_intervals = int(LENGTH / minute)
    intervals = []
    for i in range(num_of_intervals):
        intervals.append([]) # the Empty element
        startpoint = i * minute
        endpoint = (i + 1) * minute
        for j in range(len(period_points)):
            if startpoint <= period_points[j][0] and period_points[j][0] < endpoint:
                intervals[-1].append(period_points[j])
    return intervals

def find_intervals_channel(period_points, minute, message_types):
    """
    same as find_intervals. Except that it filters out text type Messages.. 
    copy and Paste Code
    """
    global LENGTH #=3600 = Eine stunde! 
    num_of_intervals = int(LENGTH / minute) # Wieviele Intervalle innerhalb einer Stunde. Weil die Period Points pro Stunde sind.. 
    intervals = []
    for i in range(num_of_intervals):
        intervals.append([])
        startpoint = i * minute
        endpoint = (i + 1) * minute
        for j in range(len(period_points)):
            if startpoint <= period_points[j][0] and period_points[j][0] < endpoint and message_types[j] != 'text':
                intervals[-1].append(period_points[j])
    return intervals

def find_matches(channel_interval, user_interval):
    """
    Bekommt einen Award für die Hässlichste Lösung bisher

    Vergleicht channel Intervals mit den Userintervals, zählt alle Matches der Events (time und size)
    Ein Match ist wenn die time (max) +- DELTA sekunden auseinanderliegt, und die Größen abweichung kleiner ist als GAMMA*Size (Also die relative größe des Bursts)

    `Return:` Die Treffer Quote (number_of_matches / all Events)
    """
    number_of_matches = 0
    number_of_nonmatches = 0
    matched_user_intervals = set()
    # Vergleicht jedes Event aus channel_interval mit jedem user_intervall (Versucht diese zu machten. Mit Gamma (Size Threshold) und Delta (Time Threshold))
    # Probiert verschiedene Deltas durch (Zeitliche Abweichungen)
    # Wenn ein Match gefunden wurde, dann wird mit dem nächsten channel_interval fortgeführt
    for j in range(min(len(channel_interval), len(user_interval))):
        time = channel_interval[j][0]
        size = channel_interval[j][1]
        is_matched = False
        for d in range(1, DELTA): # Innerhalb von 1-15 sekunden -> Geht verschiedene Delta werte durch. Zeitliche Differenz. Um uhrzeit drifft auszugleichen
            for i in range(len(user_interval)):
                if user_interval[i] in matched_user_intervals: # Wenn  schon gematcht -> Weitergehen
                    continue
                if abs(user_interval[i][0] - time) < d: # Wenn innerhalb des Deltas (welches schrittweise erhöt wird) Also Differenz der beiden Timestamps
                    if abs(size - user_interval[i][1]) < GAMMA * size: # Hmm hier wird gamma in Abhändigkei der Größe gesetzt. Relative Abweichung... (Im Paper anders )
                        matched_user_intervals.add(user_interval[i])
                        number_of_matches += 1
                        is_matched = True
                        break
            else:
                continue # Wird gemacht wenn kein Match gefunden wurde (also die schleife nicht mit Break verlassen wurde)
            break # Wird gemacht wenn ein Match gefunden wurde..
        if is_matched is False:
            number_of_nonmatches += 1
    return number_of_matches / float(number_of_matches + number_of_nonmatches)

def detect_correlations_from_intervals(channel_intervals, user_intervals):
    """
    `Return` liste mit Treffer Quote für jedes Interval.\\
        `-1` wenn keine Events in einem Interval\\
        `0` wenn nur ein Event in `channel_interval` bzw `user_interval`

    """
    correlation = []
    for i in range(len(channel_intervals)): # Was ist wenn einer der beiden weniger oder mehr Elemente hat ?! o.O
        if len(channel_intervals[i]) == 0 and len(user_intervals[i]) == 0: # no events in either one
            correlation.append(-1)
        elif len(channel_intervals[i]) == 0 or len(user_intervals[i]) == 0: # only one has an event
            correlation.append(0)
        else:
            correlation.append(find_matches(channel_intervals[i], user_intervals[i]))
    return correlation


def detect_correlations_from_bursts(channel_bursts, user_bursts, message_types, j): #Input Data is one Dataset file (one File -> One Hour)
    """
    1. Splits the Bursts into Observation Intervals (Observation Lengths are provided by `j` arg)\\
    2. Detects Matches\\
    `Return`: a Match Ratios for every Interval (as List/Array)
    """
    global observation_lengths
    correlations = []
    user_intervals = find_intervals(user_bursts, observation_lengths[j])
    channel_intervals = find_intervals_channel(channel_bursts, observation_lengths[j], message_types) # Wird wahrscheinlich benutzt weil text messages eh zu klein sind..
    correlation_of_intervals = detect_correlations_from_intervals(channel_intervals, user_intervals)
    return correlation_of_intervals


def remove_empty_correlations(correlations): # Not Used
    organized = []
    for i in range(len(correlations)):
        organized.append([c for c in correlations[i] if c != -1])
    return organized

### calculate correlations for positive samples (correlated pairs of flows)

In [ ]:
all_correlations_tp = [] # Ist dann eine Liste von Matchratio List pro File
for j in range(len(observation_lengths)): # Test Correlation_Match_Ratios for Different Obsveration Lengths
    print('For {}-second interval:'.format(observation_lengths[j]))
    corrs_tp = []
    for i in range(len(all_user_bursts)): # for every hour (every File?)
        corrs = detect_correlations_from_bursts(all_channel_bursts[i], all_user_bursts[i], message_types_of_all_senders[i], j)
        if corrs != -1:
            corrs_tp.extend(corrs)
        print ("Hour {} is done".format(i))
    all_correlations_tp.append([c for c in corrs_tp if c != -1]) # MAAAAN die haben sich doch extra eine Funktion geschrieben die alle negativen herausfiltert.. Stattdessen machen die hier so eine häßliche scheiße
with open(os.path.join(results_dir, 'corrs_tp_event_based_delta_{}.pickle'.format(DELTA)), 'wb') as handle: # Abspeichern der Match Ratios
    pickle.dump(all_correlations_tp, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Calculate correlations for negative samples (uncorrelated pairs of flows):

Geht alle Pages durch (``all_user_bursts``, ``all_channel_bursts``).. und versucht diese miteinander zu matchen. \
bei gleichen Indizes wird geskippt. \
Dadurch wird versucht events zu matchen die keine sind. -> Dadurch FP

In [ ]:
# calculate correlations for false samples (uncorrelated pairs of flows)

all_corrs_fp = []
for m in range(len(observation_lengths)):
    print('For {}-second interval:'.format(observation_lengths[m]))
    
    corrs_fp = []
#     for j in range(len(all_user_bursts[i])):
    for i in range(len(all_user_bursts)):
        for j in range(len(all_channel_bursts)):
            if i == j:
                continue # Skippt wenn gleicher Index (Also Korrelierende Files sind)
            corrs = detect_correlations_from_bursts(all_channel_bursts[j], all_user_bursts[i], message_types_of_all_senders[j], m)
            if len(corrs) == 0:
                print('warning')
    #             continue
#             organized_correlations = remove_empty_correlations(corrs)
    #         for k in range(len(organized_correlations)):
    #             corrs_fp[k].extend(organized_correlations[k])
            if corrs != -1:
                corrs_fp.extend(corrs)
            print ("user {} with channel {} is done".format(i, j))
    all_corrs_fp.append([c for c in corrs_fp if c != -1])
with open(os.path.join(results_dir, 'corrs_fp_event_based_delta_{}.pickle'.format(DELTA)), 'wb') as handle:
    pickle.dump(all_corrs_fp, handle, protocol=pickle.HIGHEST_PROTOCOL)